In [51]:
# import torch
# import pandas as pd

# # Model
# model = torch.hub.load('ultralytics/yolov5', 'yolov5x')


In [52]:
data = pd.read_csv("./normalized_data/unique_plates.csv")
sample = data.sample(1)

# paths = sample["img_path"].tolist()

# for path in paths:
#     results = model(path)
#     results.show()


In [53]:
sample = data.sample(200, random_state=123)

percent = 0.9

train = sample.iloc[:int(len(sample)*percent)]
val = sample.iloc[int(len(sample)*percent):]


In [54]:
import os


def copy(src_file, dest): return os.system(f"cp {src_file} {dest}")

for i in train.index:
    path = train["img_path"][i]

    name = path.split('/')[-1]

    copy(path, "./yolo/data/images/train/"+name+".jpg")

    with open("./yolo/data/labels/train/" + name+'.txt', 'w') as f:
        f.write("0 ")
        f.write(str(train["xmin"][i])+" ")
        f.write(str(train["ymin"][i])+" ")
        f.write(str(train["xmax"][i])+" ")
        f.write(str(train["ymax"][i]))

for i in val.index:
    path = val["img_path"][i]

    name = path.split('/')[-1]

    copy(path, "./yolo/data/images/val/"+name+".jpg")

    with open("./yolo/data/labels/val/" + name+'.txt', 'w') as f:
        f.write("0 ")
        f.write(str(val["xmin"][i])+" ")
        f.write(str(val["ymin"][i])+" ")
        f.write(str(val["xmax"][i])+" ")
        f.write(str(val["ymax"][i]))


In [55]:
# python train.py --img 640 --batch 16 --epochs 70 --data coco128.yaml --weights yolov5x.pt --cache
